In [22]:
%reset -f

import pandas as pd
import re

from my_utils import list_files_in_directory, process_cols_v2

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

def product_df():
    '''product_df docstring
    return most recent product export from bigcommerce'''
    path = ''
    files = list_files_in_directory(path)
    x = [i for i in files if 'products-' in i]
    x.sort()
    filepath = x[-1]

    df = pd.read_csv(filepath)
    df.columns = process_cols_v2(df.columns)
    return df

df = product_df()

In [26]:
ndf = df.loc[df.item_type == 'Product']
y = ndf.product_description[0]
y

'<p>D-line P3 is fast for a putter and a very torque resistant flyer. It has lower profile than normal putter and can be used as a mid-range too. It is a stable disc with predictable fade at the end. It&acute;s a hybrid between the putter and mid-range &ndash; you make the call! The P3\'s best weapon is it\'s versatility - it\'s an awesome flyer off the tee and a great tool for putting too. For the fans of fade-in putters out there this disc is a must try. We recommend this disc to players looking for a good throwing putter.</p> <p>Information about <a title="Information about this plastic type" href="https://www.wrightlife.com/plastic-types-and-descriptions/">Discmania D-Line plastic</a></p> <h3>FLIGHT CHARACTERISTICS</h3> <div><img class="__mce_add_custom__" title="speed-3.gif" src="https://cdn2.bigcommerce.com/server100/yt68tfv9/product_images/uploaded_images/speed-3.gif" alt="speed-3.gif" width="70" height="62" /><img class="__mce_add_custom__" title="glide-2.gif" src="https://cdn2

In [27]:
def flight_chars_dict(x):
    patterns = ['title="speed-(.+?)gif','title="glide-(.+?)gif','title="turn-(.+?)gif','title="fade-(.+?)gif']
    cats = ['speed','glide','turn','fade']
    d = {}
    for pattern, cat in zip(patterns,cats):
        match = re.findall(pattern,x)
        if len(match)==1:
            d[cat] = match[0].replace('.','')
    return d

def clean_titles(x):
    # extract h3 titles
    # loop through un-modified titles and sub title-case titles with h2 in place of h3
    h3_titles = re.findall('<h3>(.+?)</h3>',x)
    h3 = '<h3>{}</h3>'
    h2 = '<h2>{}</h2>'
    for i,j in zip(h3_titles,[i.title() for i in h3_titles]):
        x = re.sub(h3.format(i),h2.format(j),x)
    return x

def remove_content(x):
    # remove "Info about plastic" section
    # remove everything after more information
    regex_list = ['<h3>FLIGHT CHARACTERISTICS</h3> <div>.+?</a></p>',
                  '<p>Information about <a title="Information about this plastic type.+?</p>',
                  '<h3>More Information</h3> <p>.+'
                 ]
    for i in regex_list:
        x = re.sub(i,'',x)
    return x

def add_flight_chars(d,x):
    '''add_chars docstring
    appends bullet list of flight chars to end'''
    temp = []
    temp.append('<h2>Flight Characteristics</h2> <ul>')
    for i in d:
        temp.append('<li>{}</li>'.format(i.title()+': '+d[i]))
    temp.append('</ul>')
    temp = ' '.join(temp)
    temp = temp.replace('zero','0')
    return x+temp

d = flight_chars_dict(y)
x = remove_content(y)
x = clean_titles(x)
x = add_flight_chars(d,x)

In [28]:
x

"<p>D-line P3 is fast for a putter and a very torque resistant flyer. It has lower profile than normal putter and can be used as a mid-range too. It is a stable disc with predictable fade at the end. It&acute;s a hybrid between the putter and mid-range &ndash; you make the call! The P3's best weapon is it's versatility - it's an awesome flyer off the tee and a great tool for putting too. For the fans of fade-in putters out there this disc is a must try. We recommend this disc to players looking for a good throwing putter.</p>   <h2>Specifications</h2> <ul> <li>Classification: Putter</li> <li>Diameter: 21.2cm</li> <li>Best Choice for: Wind putter, off the tee putter, in between a putter and a mid-range</li> </ul> <h2>Flight Characteristics</h2> <ul> <li>Speed: 3</li> <li>Glide: 2</li> <li>Turn: 0</li> <li>Fade: 3</li> </ul>"